In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

In [3]:
data = pd.read_csv('/content/preprocessed_data.csv')

In [4]:
text = data['Paragraph'].values
labels = data['Topic'].values

In [5]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)


In [6]:
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
tfidf_features = tfidf_vectorizer.fit_transform(text).toarray()

In [7]:
X = np.expand_dims(tfidf_features, axis=-1)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, categorical_labels, test_size=0.2, random_state=42)


In [9]:
model = Sequential([
    Conv1D(128, kernel_size=3, activation="relu", input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(y_train.shape[1], activation="softmax")
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [14]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.4446 - loss: 1.8482 - val_accuracy: 0.7730 - val_loss: 0.7917
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.7435 - loss: 0.8657 - val_accuracy: 0.7928 - val_loss: 0.6988
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.7922 - loss: 0.6777 - val_accuracy: 0.8023 - val_loss: 0.6701
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8172 - loss: 0.5727 - val_accuracy: 0.8058 - val_loss: 0.6764
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8421 - loss: 0.4946 - val_accuracy: 0.8085 - val_loss: 0.6780
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.8527 - loss: 0.4499 - val_accuracy: 0.8050 - val_loss: 0.6936
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.8658 - loss: 0.4117 - val_accuracy: 0.8080 - val_loss: 0.7090
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.8739 - loss: 0.3806 - 

In [15]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8063 - loss: 0.9505
Test Accuracy: 0.81
